# AWS Glue Studio Notebook - Redes Sociais

##### Você está executando um Notebook do AWS Glue Studio. Para começar a utilizá-lo, você precisa iniciar um Sessão interativa do AWS Glue.

Este é o notebook utilizado no processo de ETL dos dados da Tabela Redes Sociais da Zoop. Observe cada uma das etapas e, em caso de dúvidas, assista ao vídeo correspondente ao tratamento desta tabela.

Faça bom proveito deste conteúdo! &#x1F642;

#### Opcional: Execute esta célula se quiser observar os comandos deste notebook ("magics").

In [ ]:
%help

#### Configurando e iniciando uma sessão interativa.

In [1]:
%idle_timeout 60
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 60
Session ID: 1b6a0633-95f4-4389-962d-cc228c17d01f
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session 1b6a0633-95f4-4389-962d-cc228c17d01f to get into ready status...
Session 1b6a0633-95f4-4389-962d-cc228c17d01f has be

#### Criando um DynamicFrame de uma tabela no AWS Glue Data Catalog e mostrando seu schema

In [2]:
redes_sociais_zoop_dyf = glueContext.create_dynamic_frame.from_catalog(database='db-glue-zoop', table_name='zoop-glue-redes_sociais_zoop_bronze_parquet')
redes_sociais_zoop_dyf.printSchema()

root
|-- ID_social: long
|-- Data: string
|-- Influencia_autor: long
|-- Plataforma: string
|-- Nome_produto: string
|-- Categoria_produto: string
|-- Comentario: string


#### Criando um Spark DataFrame de um DynamicFrame

In [3]:
redes_sociais_zoop_df = redes_sociais_zoop_dyf.toDF()
redes_sociais_zoop_df.show(5, truncate=False)

+---------+----------+----------------+-----------+----------------------+-----------------+---------------------------------------------------------------------------------------+
|ID_social|Data      |Influencia_autor|Plataforma |Nome_produto          |Categoria_produto|Comentario                                                                             |
+---------+----------+----------------+-----------+----------------------+-----------------+---------------------------------------------------------------------------------------+
|3933     |2022-12-02|65011           |Instagram  |Panela elétrica       |Eletrodomésticos |Atendeu todas as minhas necessidades. Recomendo! #Panelaeletrica Nota 5                |
|2858     |2022-08-28|11777           |Instagram  |Máquina de lavar roupa|Eletrodomésticos |Frustrado com a compra. Produto muito inferior ao esperado. #Maquinadelavarroupa Nota 2|
|6233     |2023-07-27|33012           |X (Twitter)|Geladeira             |Eletrodomésticos |Rec

#### Extraindo as notas da coluna `Comentario`, criando uma coluna `Avaliacao` e retirando a expressão “Nota X” da coluna `Comentario`

In [4]:
from pyspark.sql.functions import regexp_extract, regexp_replace

redes_sociais_zoop_df = redes_sociais_zoop_df.withColumn("Avaliacao", regexp_extract("Comentario", r"Nota (\d)", 1))
redes_sociais_zoop_df = redes_sociais_zoop_df.withColumn("Comentario", regexp_replace("Comentario", r" Nota \d", ""))

redes_sociais_zoop_df.show(truncate=False)

+---------+----------+----------------+-----------+----------------------+-----------------+--------------------------------------------------------------------------------+---------+
|ID_social|Data      |Influencia_autor|Plataforma |Nome_produto          |Categoria_produto|Comentario                                                                      |Avaliacao|
+---------+----------+----------------+-----------+----------------------+-----------------+--------------------------------------------------------------------------------+---------+
|3933     |2022-12-02|65011           |Instagram  |Panela elétrica       |Eletrodomésticos |Atendeu todas as minhas necessidades. Recomendo! #Panelaeletrica                |5        |
|2858     |2022-08-28|11777           |Instagram  |Máquina de lavar roupa|Eletrodomésticos |Frustrado com a compra. Produto muito inferior ao esperado. #Maquinadelavarroupa|2        |
|6233     |2023-07-27|33012           |X (Twitter)|Geladeira             |Eletro

#### Verificando os tipos dos dados convertendo o Spark DataFrame em um Pandas DataFrame

In [5]:
redes_sociais_zoop_df.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID_social          10000 non-null  int64 
 1   Data               10000 non-null  object
 2   Influencia_autor   10000 non-null  int64 
 3   Plataforma         10000 non-null  object
 4   Nome_produto       10000 non-null  object
 5   Categoria_produto  10000 non-null  object
 6   Comentario         10000 non-null  object
 7   Avaliacao          10000 non-null  object
dtypes: int64(2), object(6)
memory usage: 625.1+ KB


#### Convertendo um Spark Data Frame em um DynamicFrame

In [6]:
from awsglue.dynamicframe import DynamicFrame

redes_sociais_zoop_dyf = DynamicFrame.fromDF(redes_sociais_zoop_df, glueContext)

#### Ajustando os tipos dos dados e mapeando as colunas desejadas

In [7]:
redes_sociais_zoop_dyf_mapeado = redes_sociais_zoop_dyf.apply_mapping(
    mappings=[
        ("ID_social", "long", "id_social", "long"),
        ("Data", "string", "data", "date"),
        ("Influencia_autor", "long", "influencia_autor", "int"),
        ("Plataforma", "string", "plataforma", "string"),
        ("Nome_produto", "string", "produto", "string"),
        ("Categoria_produto", "string", "categoria_produto", "string"),
        ("Avaliacao", "string", "avaliacao", "int"),
        ("Comentario", "string", "comentario", "string")
    ]
)

redes_sociais_zoop_dyf_mapeado.printSchema()

root
|-- id_social: long
|-- data: date
|-- influencia_autor: int
|-- plataforma: string
|-- produto: string
|-- categoria_produto: string
|-- avaliacao: int
|-- comentario: string


#### Escrevendo os dados do DynamicFrame em um objeto do Bucket S3 na camada silver e em uma tabela no AWS Glue Data Catalog


In [8]:
s3output = glueContext.getSink(
  path="s3://dados-zoop/silver/redes-sociais",  # adicione o nome do seu bucket no path
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="db-glue-zoop", catalogTableName="zoop-glue-redes_sociais_zoop_silver"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(redes_sociais_zoop_dyf_mapeado)